# Pregunta 4B | Modelamiento supervisado y clasificación
## Integrantes: 
##  Juan Pablo Vergara
##  Pablo Rivera

Utilizando Apache Spark, las librerías de SparkML y el dataset créditos_bancarios.xlsx

Cree un modelo de predicción de la variable “credit_risk”. Pruebe con al menos 3 algoritmos supervisados distintos, por ej, regresión logística, árboles de decisión y random forests:

En sus resultados comente respecto de:

* A) qué tipo de preprocesamientos fue necesario realizarle a los datos, especialmente respecto de si fue necesario balancear la data

* B) Los resultados de su modelo tanto en muestra de training como de testing.

* C) Cree una tabla donde se puedan comparar las métricas de recall, precisión y accuracy para ambas clases “good” y “bad” payer (pagador). Comente y justifique cual de los 3 algoritmos funcionó mejor y por qué.

D) Realice una breve discusión de cómo se podría implementar dicho modelo en la práctica.

La descripción de los datos proporcionada, según la tabla de códigos es:

#### **Variables Predictoras**

*   **status**: 1 no checking account; 2 ... < 0 DM; 3 0<= ... < 200 DM; 4 ... >= 200 DM / salary for at least 1 year 
*   **duration**: in months
*   **credit_history**: 0 delay in paying off in the past; 1 critical account/other credits elsewhere; 2 no credits taken/all credits paid back duly; 3 existing credits paid back duly till now; 4 all credits at this bank paid back duly.
*   **purpose**: o others; 1 car (new); 2 car (used); 3 furniture/equipment; 4  radio/television; 5 domestic appliances; 6 repairs; 7 education; 8 vacation; 9 retraining; 10 business.
*   **amount**: in deutsche mark
*   **savings**: 1 unknown/no savings account; 2 ... < 100 DM ; 3 100 <= ... < 500 DM; 4 500 <= ... < 1000 DM; 5 ... >= 1000 DM
*   **employment_duration**: 1 unemployed ; 2 < 1 yr ; 3 1 <= ... < 4 yrs ; 4 4 <= ... < 7 yrs ; 5 >= 7 yrs
*   **installment_rate**: 1 >= 35; 2 25 <= ... < 35; 3 20 <= ... < 25; 4 < 20
*   **personal_status_sex**: 1 male : divorced/separated; 2 female : non-single or male : single; 3 male : married/widowed; 4 female : single.
*   **other_debtors**: 1 none; 2 co-applicant; 3 guarantor.
*   **present_residence**: 1 < 1 yr; 2 1 <= ... < 4 yrs ; 3 1 <= ... < 4 yrs 4 <= ... < 7 yrs; 4 >= 7 yrs
*   **property**: 1 unknown / no property; 2 car or other; 3 building soc. savings agr./life insurance; 4 real estate.
*   **age**: in years.
*   **other_installment_plans**: 1 bank; 2 stores; 3 none.
*   **housing**: 1 for free; 2 rent; 3 own.
*   **number_credits**: 1 1; 2 2-3; 3 4-5; 4 >= 6.
*   **job**: 1 unemployed/unskilled - non-resident; 2 unskilled - resident; 3 skilled employee/official; 4 manager/self-empl./highly qualif. employee.
*   **people_liable**: 1 3 or more; 20 to 2.
*   **telephone**: 1 no; 2 yes (under customer name).
*   **foreign_worker**: 1 yes; 0 no.

#### **Variable Objetivo**

*   **credit_risk**: 0 bad; 1 good

# Instalando Spark y sus pre-requisitos

Spark y sus prerequisitos no están preinstalados en GoogleColab.
Por lo tanto, el primer paso es instalarlos. El principal pre-requisito es java y opcional 
hadoop.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

El siguiente paso es definir las variables de entorno

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

# Configurando la Sesión y lanzando Spark

Podemos comenzar a utilizar spark. Creamos una SparkSession.

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
.master("local[2]")\
.appName("Colab")\
.config('spark.driver.memory', '10g')\
.config('spark.ui.port', '4040')\
.getOrCreate()

# Cargando los datos

Importamos pandas para poder cargar los datos de excel e iniciamos sesión en Google Drive

In [ ]:
import pandas as pd
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Cargamos el dataset utilizando pandas para posteriormente crear un dataframe de spark, definiendo las variables numéricas como enteras y las variables que representan categorías como strings.

In [ ]:
ruta_data_set = 'gdrive/My Drive/TAREA BIG DATA/creditos_bancarios.xlsx'
pd_data_set = pd.read_excel(ruta_data_set, sheet_name='SouthGermanCredit', header = 0, dtype ={"status":str, "duration":int, "credit_history":str, "purpose":str, "amount":int, "savings":str, "employment_duration":str, "installment_rate":str, "personal_status_sex":str,
                                                                                               "other_debtors":str, "present_residence":str, "property":str, "age":int, "other_installment_plans":str, "housing":str, "number_credits":str, "job":str, "people_liable":str,
                                                                                               "telephone":str, "foreign_worker":str, "credit_risk":str})
spark_data_set = spark.createDataFrame(pd_data_set)

# Exploración de Datos

Como análisis inicial de los datos, el objetivo de esta explioración es comprender qué hay en el dataset y las características de las variables.

Primero observamos una muestra inicial de los datos para asegurarnos que están bien cargados.

In [ ]:
spark_data_set = spark.createDataFrame(pd_data_set)
spark_data_set.show(10)

+------+--------+--------------+-------+------+-------+-------------------+----------------+-------------------+-------------+-----------------+--------+---+-----------------------+-------+--------------+---+-------------+---------+--------------+-----------+
|status|duration|credit_history|purpose|amount|savings|employment_duration|installment_rate|personal_status_sex|other_debtors|present_residence|property|age|other_installment_plans|housing|number_credits|job|people_liable|telephone|foreign_worker|credit_risk|
+------+--------+--------------+-------+------+-------+-------------------+----------------+-------------------+-------------+-----------------+--------+---+-----------------------+-------+--------------+---+-------------+---------+--------------+-----------+
|     1|      18|             4|      2|  1049|      1|                  2|               4|                  2|            1|                4|       2| 21|                      3|      1|             1|  3|            

In [ ]:
spark_data_set.count(),len(spark_data_set.columns)

(1000, 21)

El dataset contiene 1.000 filas u observaciones y 21 columnas o variables, de las cuales 20 son variables predictoras y 1 variable es la objetivo.

Corroboramos que el tipo de dato corresponda a la definición entregada entre variables numéricas y categóricas:

In [ ]:
spark_data_set.dtypes

[('status', 'string'),
 ('duration', 'bigint'),
 ('credit_history', 'string'),
 ('purpose', 'string'),
 ('amount', 'bigint'),
 ('savings', 'string'),
 ('employment_duration', 'string'),
 ('installment_rate', 'string'),
 ('personal_status_sex', 'string'),
 ('other_debtors', 'string'),
 ('present_residence', 'string'),
 ('property', 'string'),
 ('age', 'bigint'),
 ('other_installment_plans', 'string'),
 ('housing', 'string'),
 ('number_credits', 'string'),
 ('job', 'string'),
 ('people_liable', 'string'),
 ('telephone', 'string'),
 ('foreign_worker', 'string'),
 ('credit_risk', 'string')]

Para observar estadística descriptiva seleccionaremos las 3 variables definidas como numéricas

In [ ]:
spark_data_set.select("duration", "amount","age").summary().show()

+-------+------------------+------------------+------------------+
|summary|          duration|            amount|               age|
+-------+------------------+------------------+------------------+
|  count|              1000|              1000|              1000|
|   mean|            20.903|          3271.248|            35.542|
| stddev|12.058814452756373|2822.7517598956506|11.352670131696735|
|    min|                 4|               250|                19|
|    25%|                12|              1364|                27|
|    50%|                18|              2319|                33|
|    75%|                24|              3972|                42|
|    max|                72|             18424|                75|
+-------+------------------+------------------+------------------+



Se puede observar que ninguna contiene valores nulos. La variable duración está entre los valores 4 y 72, que quiere decir que la duración de los créditos está entre los 4 meses y los 72 meses (6 años) y posee un promedio de duración de 20 meses con montos que van desde los 250 marcos alemanes hasta los 18.424 con un promedio de 3.271.
La variable edad parte en 19 años hasta los 75 años con un pronedio de 35 años.

De lo anterior se concluye que ninguna variable presenta datos anómalos a priori.

Por otro lado, de las variables categóricas también queremos saber si existen valores perdidos:

In [ ]:
spark_data_set.summary("count").show()

+-------+------+--------+--------------+-------+------+-------+-------------------+----------------+-------------------+-------------+-----------------+--------+----+-----------------------+-------+--------------+----+-------------+---------+--------------+-----------+
|summary|status|duration|credit_history|purpose|amount|savings|employment_duration|installment_rate|personal_status_sex|other_debtors|present_residence|property| age|other_installment_plans|housing|number_credits| job|people_liable|telephone|foreign_worker|credit_risk|
+-------+------+--------+--------------+-------+------+-------+-------------------+----------------+-------------------+-------------+-----------------+--------+----+-----------------------+-------+--------------+----+-------------+---------+--------------+-----------+
|  count|  1000|    1000|          1000|   1000|  1000|   1000|               1000|            1000|               1000|         1000|             1000|    1000|1000|                   1000|

Por lo que descartamos valores perdidos en el dataset.

Otro aspecto importante es revisar si existe un **desbalanceo** de las clases de la variable objetivo, si existe un desbalanceo debemos usar una estrategia para resolver este desequilibrio.

In [ ]:
spark_data_set.groupBy("credit_risk").count().show()

+-----------+-----+
|credit_risk|count|
+-----------+-----+
|          0|  300|
|          1|  700|
+-----------+-----+



Se puede constatar que para la clase malo existen 300 observaciones y para la clase bueno existen 700, 30% y 70% respectivamente. Por lo que tenemos que balancear la data para mejorar nuestras predicciones. Esto lo haremos en los pasos siguientes.

# Preparación de Datos

Como primer paso debemos transformar las variables categóricas y crear una columna para vada valor distinto que exista en la característica que estamos codificando. Usaremos el método One Hot Encoding.

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import OneHotEncoder

Antes de ejecutar el método One Hot Encoding debemos cambiar las variables a int

In [ ]:
categoricalColumns = ['status', 'credit_history', 'purpose', 'savings', 'employment_duration', 'installment_rate', 'personal_status_sex', 'other_debtors', 'present_residence','property',
                      'other_installment_plans','housing','number_credits','job','people_liable','telephone','foreign_worker']

for i in categoricalColumns:
  spark_data_set = spark_data_set.withColumn(i, spark_data_set[i].cast(IntegerType()))

spark_data_set = spark_data_set.withColumn("credit_risk", spark_data_set["credit_risk"].cast(IntegerType()))

Ahora ejecutamos One Hot Encoding y podemos observar nuestras variables dummy

In [ ]:
for i in categoricalColumns:
  onehot = OneHotEncoder(inputCols=[i], outputCols=[i + "_dummy"])
  onehot = onehot.fit(spark_data_set)
  spark_data_set = onehot.transform(spark_data_set)

spark_data_set.printSchema()

root
 |-- status: integer (nullable = true)
 |-- duration: long (nullable = true)
 |-- credit_history: integer (nullable = true)
 |-- purpose: integer (nullable = true)
 |-- amount: long (nullable = true)
 |-- savings: integer (nullable = true)
 |-- employment_duration: integer (nullable = true)
 |-- installment_rate: integer (nullable = true)
 |-- personal_status_sex: integer (nullable = true)
 |-- other_debtors: integer (nullable = true)
 |-- present_residence: integer (nullable = true)
 |-- property: integer (nullable = true)
 |-- age: long (nullable = true)
 |-- other_installment_plans: integer (nullable = true)
 |-- housing: integer (nullable = true)
 |-- number_credits: integer (nullable = true)
 |-- job: integer (nullable = true)
 |-- people_liable: integer (nullable = true)
 |-- telephone: integer (nullable = true)
 |-- foreign_worker: integer (nullable = true)
 |-- credit_risk: integer (nullable = true)
 |-- status_dummy: vector (nullable = true)
 |-- credit_history_dummy: vec

Para los modelos de machine learning solo usaremos las variables numéricas y las variables dummy recién obtenidas, procedemos a eliminar las que no ocuparemos

In [ ]:
cols = spark_data_set.columns

for i in categoricalColumns:
  cols.remove(i)

cols.remove("credit_risk")

In [ ]:
print(cols)

['duration', 'amount', 'age', 'status_dummy', 'credit_history_dummy', 'purpose_dummy', 'savings_dummy', 'employment_duration_dummy', 'installment_rate_dummy', 'personal_status_sex_dummy', 'other_debtors_dummy', 'present_residence_dummy', 'property_dummy', 'other_installment_plans_dummy', 'housing_dummy', 'number_credits_dummy', 'job_dummy', 'people_liable_dummy', 'telephone_dummy', 'foreign_worker_dummy']


Ahora combinaremos todas las variables predictoras en un único vector llamado features

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=cols, outputCol="features")

spark_data_set = assembler.transform(spark_data_set)
spark_data_set.select("features").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                               |
+---------------------------------------------------------------------------------------------------------------------------------------+
|(70,[0,1,2,4,13,22,28,37,40,48,54,57,63,67],[18.0,1049.0,21.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                            |
|(70,[0,1,2,4,11,22,29,33,38,40,44,47,54,58,63,65,67],[9.0,2799.0,36.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])        |
|(70,[0,1,2,5,9,20,23,30,33,37,40,47,54,57,62,67],[12.0,841.0,23.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                |
|(70,[0,1,2,4,11,22,29,34,38,40,44,47,54,58,62,65,67,69],[12.0,2122.0,39.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(70,[0,1,2,4,11,22,29,38,40,48,51

Ahora escalaremos el vector feature

In [ ]:
from pyspark.ml.feature import StandardScaler

standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
spark_data_set=standardscaler.fit(spark_data_set).transform(spark_data_set)
spark_data_set.select("features","Scaled_features").show(5)

+--------------------+--------------------+
|            features|     Scaled_features|
+--------------------+--------------------+
|(70,[0,1,2,4,13,2...|(70,[0,1,2,4,13,2...|
|(70,[0,1,2,4,11,2...|(70,[0,1,2,4,11,2...|
|(70,[0,1,2,5,9,20...|(70,[0,1,2,5,9,20...|
|(70,[0,1,2,4,11,2...|(70,[0,1,2,4,11,2...|
|(70,[0,1,2,4,11,2...|(70,[0,1,2,4,11,2...|
+--------------------+--------------------+
only showing top 5 rows



### Train, Test Split

Ya que tenemos el preprocesado de los datos vamos a separar el dataset en conjunto de entrenamiento y prueba.

In [ ]:
train, test = spark_data_set.randomSplit([0.8, 0.2], seed=202301)

In [ ]:
dataset_size=float(train.select("credit_risk").count())
numNegatives=train.select("credit_risk").where('credit_risk == 0').count()

per_ones=(float(numNegatives)/float(dataset_size))*100
numPositives=float(dataset_size-numNegatives)
print('El número de bad payers {}'.format(numNegatives))
print('El porcentaje de bad payers es {}'.format(per_ones))

El número de bad payers 239
El porcentaje de bad payers es 30.368487928843713


Podemos observar que el porcentaje de bad payers es del 30%, por lo que debemos balancear nuestro set de datos.

## Balanceo

In [ ]:
BalancingRatio= numPositives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

BalancingRatio = 0.6963151207115629


Agregamos la proporción como "classWeights" al dataset para ser llamada en cada modelo

In [ ]:
from pyspark.sql.functions import when

train=train.withColumn("classWeights", when(train.credit_risk == 1,BalancingRatio).otherwise(1-BalancingRatio))
train.select("classWeights").show(5)

+------------------+
|      classWeights|
+------------------+
|0.6963151207115629|
|0.6963151207115629|
|0.6963151207115629|
|0.6963151207115629|
|0.6963151207115629|
+------------------+
only showing top 5 rows



# Modelos de Machine Learning

Los modelos supervisados que usaremos son:
* Logistic Regression
* Decision Tree
* Gradient-Boosted Tree Classifier

# Logistic Regression

El primer algoritmo que aplicaremos es Logistic Regresion. Importamos la clase para crear el Logistic Regression classifier.

In [ ]:
from pyspark.ml.classification import LogisticRegression

Creamos un objeto LogisticRegression

In [ ]:
lr = LogisticRegression(labelCol="credit_risk", featuresCol="Scaled_features",weightCol="classWeights",maxIter=100)

Ajustamos el modelo a los datos de entrenamiento

In [ ]:
lr_model=lr.fit(train)

## Evaluación

Ya entrenado el modelo, evaluamos su efectividad. Haremos la evaluación con el dataset de train y de test

In [ ]:
predict_train_lr=lr_model.transform(train)
predict_test_lr=lr_model.transform(test)

In [ ]:
# Confusion Matrix Train

print("Confusion Matrix Logistic Regression Train")
predict_train_lr.groupBy('credit_risk', 'prediction').count().show()

# Calculate the elements of the confusion matrix

TP_train_lr = predict_train_lr.filter('prediction = 1 AND credit_risk = prediction').count()
FP_train_lr = predict_train_lr.filter('prediction = 1 AND credit_risk = 0').count()
FN_train_lr = predict_train_lr.filter('prediction = 0 AND credit_risk = 1').count()
TN_train_lr = predict_train_lr.filter('prediction = 0 AND credit_risk = prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy_train_lr = round((TN_train_lr + TP_train_lr) / (TN_train_lr + TP_train_lr + FN_train_lr + FP_train_lr),2)
print("accuracy LR:", accuracy_train_lr)

# Recall: Número de elementos identificados correctamente como positivos del total de positivos verdaderos
recall_train_lr = round(TP_train_lr / (TP_train_lr + FN_train_lr),2)
print("recall LR:", recall_train_lr)

recall_train_neg_lr = round(TN_train_lr / (TN_train_lr + FP_train_lr),2)
print("recall N LR:", recall_train_neg_lr)

# Precision: Número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.
precision_train_lr = round(TP_train_lr / (TP_train_lr + FP_train_lr),2)
print("precision LR:", precision_train_lr)

precision_train_neg_lr = round(TN_train_lr / (TN_train_lr + FN_train_lr),2)
print("precision N LR:", precision_train_neg_lr)


Confusion Matrix Logistic Regression Train
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    7|
|          0|       0.0|   85|
|          1|       1.0|  541|
|          0|       1.0|  154|
+-----------+----------+-----+

accuracy LR: 0.8
recall LR: 0.99
recall N LR: 0.36
precision LR: 0.78
precision N LR: 0.92


In [ ]:
# Confusion Matrix Test

print("Confusion Matrix Logistic Regression Test")
predict_test_lr.groupBy('credit_risk', 'prediction').count().show()

# Calculate the elements of the confusion matrix

TP_lr = predict_test_lr.filter('prediction = 1 AND credit_risk = prediction').count()
FP_lr = predict_test_lr.filter('prediction = 1 AND credit_risk = 0').count()
FN_lr = predict_test_lr.filter('prediction = 0 AND credit_risk = 1').count()
TN_lr = predict_test_lr.filter('prediction = 0 AND credit_risk = prediction').count()

# Accuracy proporción de predicciones correctas
accuracy_lr = round((TN_lr + TP_lr) / (TN_lr + TP_lr + FN_lr + FP_lr),2)
print("accuracy LR:", accuracy_lr)

# Recall: Número de elementos identificados correctamente como positivos del total de positivos verdaderos
recall_lr = round(TP_lr / (TP_lr + FN_lr),2)
print("recall LR:", recall_lr)

recall_neg_lr = round(TN_lr / (TN_lr + FP_lr),2)
print("recall N LR:", recall_neg_lr)

# Precision: Número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.
precision_lr = round(TP_lr / (TP_lr + FP_lr),2)
print("precision LR:", precision_lr)

precision_neg_lr = round(TN_lr / (TN_lr + FN_lr),2)
print("precision N LR:", precision_neg_lr)

Confusion Matrix Logistic Regression Test
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    5|
|          0|       0.0|   14|
|          1|       1.0|  147|
|          0|       1.0|   47|
+-----------+----------+-----+

accuracy LR: 0.76
recall LR: 0.97
recall N LR: 0.23
precision LR: 0.76
precision N LR: 0.74


# Decision Tree

El segundo algoritmo que aplicaremos es Decision Tree.
Importamos la clase para crear el Decision Tree classifier.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

Creamos un objeto DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(labelCol="credit_risk", featuresCol="Scaled_features",weightCol="classWeights")

Ajustamos el modelo a los datos de entrenamiento

In [ ]:
tree_model = tree.fit(train)

## Evaluación

Ahora que se ha entrenado el modelo, podemos evaluar qué tan efectivo es. Hacemos predicciones en el conjunto de train y  test, y lo comparamos con los valores conocidos.

In [ ]:
predict_train_dt = tree_model.transform(train)
predict_test_dt = tree_model.transform(test)

In [ ]:
# Confusion Matrix Train

print("Confusion Matrix Decision Tree Train")
predict_train_dt.groupBy('credit_risk', 'prediction').count().show()

# Calculate the elements of the confusion matrix

TP_train_dt = predict_train_dt.filter('prediction = 1 AND credit_risk = prediction').count()
FP_train_dt = predict_train_dt.filter('prediction = 1 AND credit_risk = 0').count()
FN_train_dt = predict_train_dt.filter('prediction = 0 AND credit_risk = 1').count()
TN_train_dt = predict_train_dt.filter('prediction = 0 AND credit_risk = prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy_train_dt = round((TN_train_dt + TP_train_dt) / (TN_train_dt + TP_train_dt + FN_train_dt + FP_train_dt),2)
print("accuracy DT:", accuracy_train_dt)

# Recall: Número de elementos identificados correctamente como positivos del total de positivos verdaderos
recall_train_dt = round(TP_train_dt / (TP_train_dt + FN_train_dt),2)
print("recall DT:", recall_train_dt)

recall_train_neg_dt = round(TN_train_dt / (TN_train_dt + FP_train_dt),2)
print("recall N DT:", recall_train_neg_dt)


# Precision: Número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.
precision_train_dt = round(TP_train_dt / (TP_train_dt + FP_train_dt),2)
print("precision DT:", precision_train_dt)

precision_train_neg_dt = round(TN_train_dt / (TN_train_dt + FN_train_dt),2)
print("precision N DT:", precision_train_neg_dt)

Confusion Matrix Decision Tree Train
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    6|
|          0|       0.0|   65|
|          1|       1.0|  542|
|          0|       1.0|  174|
+-----------+----------+-----+

accuracy DT: 0.77
recall DT: 0.99
recall N DT: 0.27
precision DT: 0.76
precision N DT: 0.92


In [ ]:
# Confusion Matrix

print("Confusion Matrix Decision Tree Test")
predict_test_dt.groupBy('credit_risk', 'prediction').count().show()

# Calculate the elements of the confusion matrix

TP_dt = predict_test_dt.filter('prediction = 1 AND credit_risk = prediction').count()
FP_dt = predict_test_dt.filter('prediction = 1 AND credit_risk = 0').count()
FN_dt = predict_test_dt.filter('prediction = 0 AND credit_risk = 1').count()
TN_dt = predict_test_dt.filter('prediction = 0 AND credit_risk = prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy_dt = round((TN_dt + TP_dt) / (TN_dt + TP_dt + FN_dt + FP_dt),2)
print("accuracy DT:", accuracy_dt)

# Recall: Número de elementos identificados correctamente como positivos del total de positivos verdaderos
recall_dt = round(TP_dt / (TP_dt + FN_dt),2)
print("recall DT:", recall_dt)

recall_neg_dt = round(TN_dt / (TN_dt + FP_dt),2)
print("recall N DT:", recall_neg_dt)


# Precision: Número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.
precision_dt = round(TP_dt / (TP_dt + FP_dt),2)
print("precision DT:", precision_dt)

precision_neg_dt = round(TN_dt / (TN_train_dt + FN_dt),2)
print("precision N DT:", precision_neg_dt)

Confusion Matrix Decision Tree Test
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    4|
|          0|       0.0|   12|
|          1|       1.0|  148|
|          0|       1.0|   49|
+-----------+----------+-----+

accuracy DT: 0.75
recall DT: 0.97
recall N DT: 0.2
precision DT: 0.75
precision N DT: 0.17


# Gradient-Boosted Tree Classifier

In [ ]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol='Scaled_features', labelCol='credit_risk', maxIter=100, weightCol="classWeights")
gbtModel = gbt.fit(train)

predict_train_gbt = gbtModel.transform(train)
predict_test_gbt = gbtModel.transform(test)

In [ ]:
# Confusion Matrix Train

print("Confusion Matrix Train")
predict_train_gbt.groupBy('credit_risk', 'prediction').count().show()

# Calculate the elements of the confusion matrix

TP_train_gbt = predict_train_gbt.filter('prediction = 1 AND credit_risk = prediction').count()
FP_train_gbt = predict_train_gbt.filter('prediction = 1 AND credit_risk = 0').count()
FN_train_gbt = predict_train_gbt.filter('prediction = 0 AND credit_risk = 1').count()
TN_train_gbt = predict_train_gbt.filter('prediction = 0 AND credit_risk = prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy_train_gbt = round((TN_train_gbt + TP_train_gbt) / (TN_train_gbt + TP_train_gbt + FN_train_gbt + FP_train_gbt),2)
print("accuracy GBT:", accuracy_train_gbt)

# Recall: Número de elementos identificados correctamente como positivos del total de positivos verdaderos
recall_train_gbt = round(TP_train_gbt / (TP_train_gbt + FN_train_gbt),2)
print("recall GBT:", recall_train_gbt)

recall_train_neg_gbt = round(TN_train_gbt / (TN_train_gbt + FP_train_gbt),2)
print("recall N GBT:", recall_train_neg_gbt)


# Precision: Número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.
precision_train_gbt = round(TP_train_gbt / (TP_train_gbt + FP_train_gbt),2)
print("precision GBT:", precision_train_gbt)

precision_train_neg_gbt = round(TN_train_gbt / (TN_train_gbt + FN_train_gbt),2)
print("precision N GBT:", precision_train_neg_gbt)

Confusion Matrix Train
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          0|       0.0|  223|
|          1|       1.0|  548|
|          0|       1.0|   16|
+-----------+----------+-----+

accuracy GBT: 0.98
recall GBT: 1.0
recall N GBT: 0.93
precision GBT: 0.97
precision N GBT: 1.0


In [ ]:
# Confusion Matrix test

print("Confusion Matrix Test")
predict_test_gbt.groupBy('credit_risk', 'prediction').count().show()

# Calculate the elements of the confusion matrix

TP_gbt = predict_test_gbt.filter('prediction = 1 AND credit_risk = prediction').count()
FP_gbt = predict_test_gbt.filter('prediction = 1 AND credit_risk = 0').count()
FN_gbt = predict_test_gbt.filter('prediction = 0 AND credit_risk = 1').count()
TN_gbt = predict_test_gbt.filter('prediction = 0 AND credit_risk = prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy_gbt = round((TN_gbt + TP_gbt) / (TN_gbt + TP_gbt + FN_gbt + FP_gbt),2)
print("accuracy GBT:", accuracy_gbt)

# Recall: Número de elementos identificados correctamente como positivos del total de positivos verdaderos
recall_gbt = round(TP_gbt / (TP_gbt + FN_gbt),2)
print("recall GBT:", recall_gbt)

recall_neg_gbt = round(TN_gbt / (TN_gbt + FP_gbt),2)
print("recall N GBT:", recall_neg_gbt)


# Precision: Número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.
precision_gbt = round(TP_gbt / (TP_gbt + FP_gbt),2)
print("precision GBT:", precision_gbt)

precision_neg_gbt = round(TN_gbt / (TN_gbt + FN_gbt),2)
print("precision N GBT:", precision_neg_gbt)

Confusion Matrix Test
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|   15|
|          0|       0.0|   32|
|          1|       1.0|  137|
|          0|       1.0|   29|
+-----------+----------+-----+

accuracy GBT: 0.79
recall GBT: 0.9
recall N GBT: 0.52
precision GBT: 0.83
precision N GBT: 0.68


## Desarrollo

A) Qué tipo de preprocesamientos fue necesario realizarle a los datos, especialmente respecto de si fue necesario balancear la data

- En la fase de preprocesamiento lo que se realizó fue un One-hot encoding dado que teníamos variables categóricas, esto fue para entregar un formato de variables adecuado a los modelos de machine learning ejecutados.

- Como pudimos observar en la data entregada es que en la variable objetivo "credit risk" existía un desbalance donde 300 observaciones pertenecían a la clase "bad" y para la clase "good" pertenecían 700 observaciones. Por lo que los datos de "bad" eran más pequeños en comparación a los "good" afectando la calidad de los modelos que construimos. Para subsanar este desbalance se decidió ejecutar los siguientes pasos:


1.   **Calcular la relación**, calculando la relación de la clase "bad" del total de observaciones
2.   **Calcular el peso**, posteriormente calculamos el peso de cada etiqueta.
3.   **Agregar columna de peso a DataFrame**, agregamos la columna calculada al DF.
4.   Habilitar la columna de peso en el algoritmo, agregándola a cada modelo ejecutado.

B) Los resultados de su modelo tanto en muestra de training como de testing.

Para ver los resultados examinaremos las matrices de confusión tanto para la muestra de entrenamiento como de testeo.

1. Logistic Regression

In [ ]:
# Confusion Matrix Train
print("Confusion Matrix Logistic Regression Train")
predict_train_lr.groupBy('credit_risk', 'prediction').count().show()

# Confusion Matrix Test
print("Confusion Matrix Logistic Regression Test")
predict_test_lr.groupBy('credit_risk', 'prediction').count().show()

Confusion Matrix Logistic Regression Train
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    7|
|          0|       0.0|   85|
|          1|       1.0|  541|
|          0|       1.0|  154|
+-----------+----------+-----+

Confusion Matrix Logistic Regression Test
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    5|
|          0|       0.0|   14|
|          1|       1.0|  147|
|          0|       1.0|   47|
+-----------+----------+-----+



Se puede observar que para la muestra train el modelo logistic regression hace 626 predicciones correctas de un total de 787 (80%). Mientras que para el conjunto de test logra 161 predicciones correctas de un total de 213 (76%). 

2. Decision Tree

In [ ]:
# Confusion Matrix Train
print("Confusion Matrix Decision Tree Train")
predict_train_dt.groupBy('credit_risk', 'prediction').count().show()

# Confusion Matrix Test
print("Confusion Matrix Decision Tree Test")
predict_test_dt.groupBy('credit_risk', 'prediction').count().show()

Confusion Matrix Decision Tree Train
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    6|
|          0|       0.0|   65|
|          1|       1.0|  542|
|          0|       1.0|  174|
+-----------+----------+-----+

Confusion Matrix Decision Tree Test
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|    4|
|          0|       0.0|   12|
|          1|       1.0|  148|
|          0|       1.0|   49|
+-----------+----------+-----+



Para este segundo caso se puede observar que para la muestra train el modelo decision tree hace 607 predicciones correctas de un total de 787 (77%). Mientras que para el conjunto de test logra 160 predicciones correctas de un total de 213 (75%). Los resultados son muy parecidos a los anteriores en la muestra de test.

3. Gradient-Boosted Tree Classifier

In [ ]:
# Confusion Matrix Train
print("Confusion Matrix Train")
predict_train_gbt.groupBy('credit_risk', 'prediction').count().show()

# Confusion Matrix test
print("Confusion Matrix Test")
predict_test_gbt.groupBy('credit_risk', 'prediction').count().show()

Confusion Matrix Train
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          0|       0.0|  223|
|          1|       1.0|  548|
|          0|       1.0|   16|
+-----------+----------+-----+

Confusion Matrix Test
+-----------+----------+-----+
|credit_risk|prediction|count|
+-----------+----------+-----+
|          1|       0.0|   15|
|          0|       0.0|   32|
|          1|       1.0|  137|
|          0|       1.0|   29|
+-----------+----------+-----+



Para este tercer modelo se puede observar que para la muestra train el modelo Gradient-Boosted Tree Classifier hace 771 predicciones correctas de un total de 787 (98%). Mientras que para el conjunto de test logra 169 predicciones correctas de un total de 213 (79%). Por sobre los dos modelos anteriores.

C) Cree una tabla donde se puedan comparar las métricas de recall, precisión y accuracy para ambas clases “good” y “bad” payer (pagador). Comente y justifique cual de los 3 algoritmos funcionó mejor y por qué.

In [ ]:
print("Accuracy")
print("----------------------------------------------------------")
print("Logistic Regression: ",accuracy_lr)
print("Decision Tree: ",accuracy_dt)
print("Gradient-Boosted Tree Classifier: ",accuracy_gbt)

print("")

print("Recall good payer")
print("----------------------------------------------------------")
print("Recall Logistic Regression:", recall_lr)
print("Recall Decision Tree:", recall_dt)
print("Recall Gradient-Boosted Tree Classifier:", recall_gbt)

print("")

print("Recall bad payer")
print("----------------------------------------------------------")
print("Recall Neg Logistic Regression:", recall_neg_lr)
print("Recall Neg Decision Tree:", recall_neg_dt)
print("Recall Neg Gradient-Boosted Tree Classifier:", recall_neg_gbt)

print("")

print("Precision good payer")
print("----------------------------------------------------------")
print("Precision Logistic Regression:", precision_lr)
print("Precision Decision Tree:", precision_dt)
print("Precision Gradient-Boosted Tree Classifier:", precision_gbt)

print("")

print("Precision bad payer")
print("----------------------------------------------------------")
print("Precision Neg Logistic Regression:", precision_neg_lr)
print("Precision Neg Decision Tree:", precision_neg_dt)
print("Precision Neg Gradient-Boosted Tree Classifier:", precision_neg_gbt)

Accuracy
----------------------------------------------------------
Logistic Regression:  0.76
Decision Tree:  0.75
Gradient-Boosted Tree Classifier:  0.79

Recall good payer
----------------------------------------------------------
Recall Logistic Regression: 0.97
Recall Decision Tree: 0.97
Recall Gradient-Boosted Tree Classifier: 0.9

Recall bad payer
----------------------------------------------------------
Recall Neg Logistic Regression: 0.23
Recall Neg Decision Tree: 0.2
Recall Neg Gradient-Boosted Tree Classifier: 0.52

Precision good payer
----------------------------------------------------------
Precision Logistic Regression: 0.76
Precision Decision Tree: 0.75
Precision Gradient-Boosted Tree Classifier: 0.83

Precision bad payer
----------------------------------------------------------
Precision Neg Logistic Regression: 0.74
Precision Neg Decision Tree: 0.17
Precision Neg Gradient-Boosted Tree Classifier: 0.68


* Si observamos el accuracy de los modelos podemos verificar que todos se sitúan por sobre el 75% de exactitud llegando a 79% el modelo de Gradient-Booested Tree Classifier. Si miramos el recall, el modelo que menos identifica los good players es este último, sin embargo para los bad players dobla a los otros dos modelos (Logistic Regression y Decision Tree). Por último, si observamos el precision, la calidad para good payer es superior para el modelo de Gradient-Boosted Decision Tree y segundo para los bad payer, superado solo por la Logistc Regression. De lo anterior se desprende que el modelo Gradient-Boosted Tree Classifier es el mejor modelo para predecir los bad y good payers.
* Este resultado se debe a que un modelo Gradient-Boosted está formado por un conjunto de árboles de decisión individuales, entrenados de forma secuencial, por lo que cada nuevo árbol trata de mejorar los errores de los árboles anteriores, es por eso que los resultados son mejores a los de Decision Tree. Además son capaces de seleccionar predictores de forma automática por lo que deja fuera variables no significativas o con bajo poder explicativo. Por último, mencionar que poseen una buena escalabilidad pudiendo aplicarse a conjuntos de datos con un elevado número de observaciones como es el caso de Big Data.

D) Realice una breve discusión de cómo se podría implementar dicho modelo en la práctica.

Primero, la entidad que otorga los créditos debe tener un sistema que capture los datos propios del préstamo así como información del cliente, por ejemplo: la duración del crédito, el propósito, el monto del crédito, antigüedad laboral.
Así como información externa a la entidad, como deuda total o cuánto es su ratio de inversión en total en el sistema bancario.

Respecto al modelo de machine learning, para pasar de una etapa de desarrollo a una etapa de producción podríamos usar Databricks que posee una plataforma MLOps que proporciona un entorno de gestión de modelos junta a otras capacidades como exploración iterativa de datos, gestión de modelos, etc.

Especificamente:

* Seguimiento: Registro y consulta de experimentos
* Proyectos: Formato de empaquetado para reproducir modelos
* Modelos: formato general de modelos para portabilidad y despliegues flexibles
* Registro: Gestión del ciclo de vida de los modelos